In [ ]:
# Please check better documented notebook in folder 'notebooks_ref/demo_complete_flow.ipynb'

In [1]:
# Standard imports
import json
import asyncio
import os
from pathlib import Path

# Rich for beautiful output
from rich.console import Console
from rich.panel import Panel
from rich.table import Table
from rich.syntax import Syntax
from rich.markdown import Markdown

# Our project imports
from support_copilot_host.models import Ticket, Trace
from support_copilot_host.orchestrator import run_ticket_flow
from support_copilot_host.agents import TriageAgent, ResearchAgent, ActionAgent, SupervisorAgent
from support_copilot_host.mcp_client import SupportMCPClient
from support_copilot_host.observability import render_trace_to_stdout, print_trace_summary

console = Console()
console.print("[bold green]✓ All imports successful![/bold green]")

✓ All imports successful!

In [2]:
# Verify environment
api_key = os.getenv("OPENAI_API_KEY")
if api_key:
    console.print(f"[green]✓ OpenAI API key found ({api_key[:10]}...)[/green]")
else:
    console.print("[red]✗ OPENAI_API_KEY not set![/red]")
    console.print("Set it with: export OPENAI_API_KEY='your-key'")

✓ OpenAI API key found (sk-proj-Ln...)

In [3]:


# Load sample tickets
with open("data/tickets/samples.json") as f:
    sample_tickets = json.load(f)

console.print(f"[cyan]Loaded {len(sample_tickets)} sample tickets[/cyan]")
for ticket in sample_tickets:
    console.print(f"  • {ticket['id']}: {ticket['description'][:60]}...")

Loaded 4 sample tickets

• TICKET-001: Customer cannot export their main workspace dashboard to CSV...

• TICKET-002: Several SSO users in the EU are failing to log in via SAML. ...

• TICKET-003: User reports they are an editor on a workspace but cannot ed...

• TICKET-004: Customer's export API integration started failing with HTTP ...

In [4]:
sample_tickets[0]

{'id': 'TICKET-001',
 'description': "Customer cannot export their main workspace dashboard to CSV. They see 'Export failed: timeout' after ~30 seconds.",
 'screenshot_path': 'data/tickets/screenshots/export_timeout.png',
 'log_snippet': '2025-11-01T09:21:10Z ERR_EXPORT_TIMEOUT workspace_id=ws_1234 service=export_service'}

# Let's do DEMO for MCP Server Tools

In [5]:
# Connect to MCP server
mcp_client = SupportMCPClient()

async def connect_mcp():
    await mcp_client.connect()
    console.print("[green]✓ Connected to MCP server![/green]")

await connect_mcp()

✓ Connected to MCP server!

In [6]:
# Search for export-related documentation
docs_results = await mcp_client.call_support_docs_search(
    query="export csv timeout",
    max_results=3
)

console.print("\n[bold cyan]Support Docs Search Results:[/bold cyan]")
for doc in docs_results:
    console.print(Panel(
        f"[yellow]Score: {doc['score']}[/yellow]\n\n{doc['snippet']}",
        title=f"[bold]{doc['title']}[/bold]",
        subtitle=doc['path'],
        border_style="cyan"
    ))

Support Docs Search Results:

╭───────────────────────────────────────────── Export to CSV Errors ──────────────────────────────────────────────╮
│ Score: 15                                                                                                       │
│                                                                                                                 │
│ # Export to CSV Errors                                                                                          │
│                                                                                                                 │
│ When users attempt to export reports to CSV and see errors like                                                 │
│ **"Export failed: timeout"** or **"Export failed: invalid workspace"**, follow                                  │
│ these steps:                                                                                                    │
│                                                                                                                 │
│ 1. Confirm the user has the `export_reports` permission on the workspace.                                       │
│ 2. Check recent incidents related to the `exp...                                                                │
╰───────────────────────────────────── docs/runbooks/export_to_csv_errors.md ─────────────────────────────────────╯

╭───────────────────────────────────────────── Logs Inspection Guide ─────────────────────────────────────────────╮
│ Score: 5                                                                                                        │
│                                                                                                                 │
│ ...log snippet:                                                                                                 │
│                                                                                                                 │
│ 1. Look for error codes (e.g., `ERR_EXPORT_TIMEOUT`, `ERR_AUTH_INVALID_TOKEN`).                                 │
│ 2. Match error codes to known runbooks such as **Export to CSV Errors** or                                      │
│    **Authentication Failures**.                                                                                 │
│ 3. Pay attention to correlation IDs and timestamps; these may be needed if you                                  │
│    escalate to engin...                                                                                         │
╰──────────────────────────────────── docs/runbooks/logs_inspection_guide.md ─────────────────────────────────────╯

In [7]:
# Search for export-related incidents
incident_results = await mcp_client.call_incidents_search(
    query="export timeout",
    max_results=3,
    status_filter=["Investigating", "Mitigating"]  # Only active incidents
)

console.print("\n[bold cyan]Incident Search Results:[/bold cyan]")
for incident in incident_results:
    status_color = "red" if incident['status'] == "Investigating" else "yellow"
    console.print(Panel(
        f"[{status_color}]Status: {incident['status']}[/{status_color}]\n\n"
        f"{incident['summary']}\n\n"
        f"[dim]Matched tags: {', '.join(incident['matched_tags'])}[/dim]",
        title=f"[bold]{incident['incident_id']}: {incident['title']}[/bold]",
        border_style="yellow"
    ))

Incident Search Results:

╭───────────────────────────── INC-1001: Export to CSV timeouts for large workspaces ─────────────────────────────╮
│ Status: Mitigating                                                                                              │
│                                                                                                                 │
│ Some customers with very large workspaces are experiencing timeouts when exporting dashboards to CSV. The       │
│ export_service is partially degraded while we roll out a fix.                                                   │
│                                                                                                                 │
│ Matched tags: export, timeout                                                                                   │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

# Let's process complete Ticket, with agents

In [8]:
# Load TICKET-001
ticket_data = sample_tickets[0]  # First sample
ticket = Ticket(**ticket_data)

console.print("\n[bold magenta]Processing Ticket[/bold magenta]")
console.print(Panel(
    f"[yellow]ID:[/yellow] {ticket.id}\n\n"
    f"[yellow]Description:[/yellow] {ticket.description}\n\n"
    f"[yellow]Log Snippet:[/yellow]\n{ticket.log_snippet}",
    title="[bold]Ticket Details[/bold]",
    border_style="blue"
))

Processing Ticket

╭──────────────────────────────────────────────── Ticket Details ─────────────────────────────────────────────────╮
│ ID: TICKET-001                                                                                                  │
│                                                                                                                 │
│ Description: Customer cannot export their main workspace dashboard to CSV. They see 'Export failed: timeout'    │
│ after ~30 seconds.                                                                                              │
│                                                                                                                 │
│ Log Snippet:                                                                                                    │
│ 2025-11-01T09:21:10Z ERR_EXPORT_TIMEOUT workspace_id=ws_1234 service=export_service                             │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [9]:
# Run the complete flow (this may take 10-20 seconds)
console.print("\n[bold cyan]Running multi-agent flow...[/bold cyan]")
console.print("[dim](This will take 10-20 seconds - 3 LLM calls + 3 tool calls)[/dim]\n")

supervisor_output, trace = await run_ticket_flow(ticket)

console.print("[bold green]✓ Flow complete![/bold green]\n")

Running multi-agent flow...

(This will take 10-20 seconds - 4 LLM calls + 3 tool calls)

✓ Flow complete!

In [10]:
# Display customer reply
console.print(Panel(
    supervisor_output.final_customer_reply,
    title="[bold green]Customer Reply[/bold green]",
    border_style="green",
    padding=(1, 2)
))

╭──────────────────────────────────────────────── Customer Reply ─────────────────────────────────────────────────╮
│                                                                                                                 │
│  Thank you for reaching out! It seems you've encountered the 'Export failed: timeout' error. This can happen    │
│  if your workspace dashboard has a lot of information, causing it to take longer to export. However, you're     │
│  not alone in this; there's an ongoing incident (INC-1001) affecting others with similar export timeout         │
│  issues. Here are a few steps you can try: 1. Check if your dashboard has too much data and consider            │
│  simplifying it. 2. If the issue persists, please monitor our status page for updates. 3. You can also refer    │
│  to our documentation on Export to CSV Errors for more information. Thank you for your understanding!           │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [11]:
# Print trace summary
print_trace_summary(trace)

Trace Summary

Trace ID: 48288906-246b-4770-a965-20d13d356251

Ticket ID: TICKET-001

Total Events: 11

Agents Involved: ActionAgent, ResearchAgent, SupervisorAgent, TriageAgent

LLM Calls: 3

Tool Calls: 2

In [12]:
# Show full trace as table
render_trace_to_stdout(trace)

                         Trace: 48288906-246b-4770-a965-20d13d356251 (Ticket: TICKET-001)                          
┏━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Timestamp    ┃ Agent           ┃ Step Type    ┃ Detail                                                          ┃
┡━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ 16:33:20.766 │ TriageAgent     │ LLM_CALL     │ Analyzing ticket and creating triage plan                       │
│ 16:33:24.221 │ TriageAgent     │ AGENT_OUTPUT │ Classified as POSSIBLE_BUG, will use tools:                     │
│              │                 │              │ support_docs.search, incidents.searc...                         │
│ 16:33:24.221 │ -               │ TOOL_CALL    │ Calling tool: support_docs.search                               │
│ 16:33:24.224 │ -               │ TOOL_RESULT  │ Tool result: support_docs.search                                │
│ 16:33:24.224 │ -               │ TOOL_CALL    │ Calling tool: incidents.search                                  │
│ 16:33:24.225 │ -               │ TOOL_RESULT  │ Tool result: incidents.search                                   │
│ 16:33:24.225 │ ResearchAgent   │ AGENT_OUTPUT │ Gathered 3 docs, 3 incidents, 0 status checks                   │
│ 16:33:24.225 │ ActionAgent     │ LLM_CALL     │ Drafting customer reply and internal note                       │
│ 16:33:31.528 │ ActionAgent     │ AGENT_OUTPUT │ Drafted customer reply and internal note                        │
│ 16:33:31.528 │ SupervisorAgent │ LLM_CALL     │ Reviewing action output for quality and safety                  │
│ 16:33:45.161 │ SupervisorAgent │ AGENT_OUTPUT │ Review complete: Edited                                         │
└──────────────┴─────────────────┴──────────────┴─────────────────────────────────────────────────────────────────┘

In [14]:
for event in trace.events:
    print(event)

timestamp='2025-11-15T16:33:20.766718Z' agent_name='TriageAgent' step_type='LLM_CALL' detail='Analyzing ticket and creating triage plan' payload={'ticket_id': 'TICKET-001'}
timestamp='2025-11-15T16:33:24.221495Z' agent_name='TriageAgent' step_type='AGENT_OUTPUT' detail='Classified as POSSIBLE_BUG, will use tools: support_docs.search, incidents.search' payload=None
timestamp='2025-11-15T16:33:24.221522Z' agent_name=None step_type='TOOL_CALL' detail='Calling tool: support_docs.search' payload={'query': "Customer cannot export their main workspace dashboard to CSV. They see 'Export failed: timeout' after ~30 seconds."}
timestamp='2025-11-15T16:33:24.224552Z' agent_name=None step_type='TOOL_RESULT' detail='Tool result: support_docs.search' payload={'count': 3, 'top_title': 'Export to CSV Errors'}
timestamp='2025-11-15T16:33:24.224561Z' agent_name=None step_type='TOOL_CALL' detail='Calling tool: incidents.search' payload={'query': "Customer cannot export their main workspace dashboard to CS

In [ ]:
timestamp='2025-11-15T16:33:24.221495Z' agent_name='TriageAgent' step_type='AGENT_OUTPUT' detail='Classified as POSSIBLE_BUG, will use tools: support_docs.search, incidents.search' payload=None


# Test also, individual agents

In [15]:
# Create a new ticket for testing
test_ticket = Ticket(
    id="TEST-001",
    description="User cannot log in via SSO. Error says 'SAML assertion expired'.",
    log_snippet="2025-11-14T10:00:00Z ERR_AUTH_SAML service=auth_service",
    screenshot_path=None
)

# Create trace
test_trace = Trace(trace_id="test-001", ticket_id="TEST-001")

# Run triage
triage_agent = TriageAgent()
triage_plan = await triage_agent.run(test_ticket, test_trace)

console.print("\n[bold cyan]Triage Plan:[/bold cyan]")
console.print(Panel(
    f"[yellow]Issue Type:[/yellow] {triage_plan.issue_type}\n\n"
    f"[yellow]Summary:[/yellow] {triage_plan.issue_summary}\n\n"
    f"[yellow]Tools to Call:[/yellow] {', '.join(triage_plan.tools_to_call)}\n\n"
    f"[yellow]Notes:[/yellow] {triage_plan.notes}",
    title="TriageAgent Output",
    border_style="blue"
))

Triage Plan:

╭────────────────────────────────────────────── TriageAgent Output ───────────────────────────────────────────────╮
│ Issue Type: CONFIG_ERROR                                                                                        │
│                                                                                                                 │
│ Summary: User is unable to log in via SSO due to an expired SAML assertion error.                               │
│                                                                                                                 │
│ Tools to Call: support_docs.search                                                                              │
│                                                                                                                 │
│ Notes: The error suggests a possible configuration issue with SSO settings related to SAML assertion            │
│ expiration.                                                                                                     │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [16]:
# Run research (uses the plan from triage)
research_agent = ResearchAgent()
research_report = await research_agent.run(
    test_ticket,
    triage_plan,
    mcp_client,
    test_trace
)

console.print("\n[bold cyan]Research Report:[/bold cyan]")
console.print(Panel(
    research_report.summary,
    title="Research Summary",
    border_style="green"
))

if research_report.docs_results:
    console.print(f"\n[cyan]Found {len(research_report.docs_results)} documentation entries[/cyan]")
    for doc in research_report.docs_results:
        console.print(f"  • {doc['title']}")

if research_report.incident_results:
    console.print(f"\n[cyan]Found {len(research_report.incident_results)} incidents[/cyan]")
    for inc in research_report.incident_results:
        console.print(f"  • {inc['incident_id']}: {inc['title']} ({inc['status']})")

if research_report.status_results:
    console.print(f"\n[cyan]Checked {len(research_report.status_results)} services[/cyan]")
    for svc in research_report.status_results:
        console.print(f"  • {svc['service_name']}: {svc['status']}")

Research Report:

╭─────────────────────────────────────────────── Research Summary ────────────────────────────────────────────────╮
│ Found 3 relevant documentation entries:                                                                         │
│   - Authentication Failures                                                                                     │
│   - Workspace Permissions Issues                                                                                │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Found 3 documentation entries

• Authentication Failures

• Workspace Permissions Issues

• Logs Inspection Guide

In [17]:
# Run action agent
action_agent = ActionAgent()
action_output = await action_agent.run(
    test_ticket,
    triage_plan,
    research_report,
    test_trace
)

console.print("\n[bold cyan]Action Agent Drafts:[/bold cyan]")
console.print(Panel(
    action_output.customer_reply,
    title="[green]Customer Reply (Draft)[/green]",
    border_style="green"
))
console.print(Panel(
    action_output.internal_note,
    title="[yellow]Internal Note (Draft)[/yellow]",
    border_style="yellow"
))

Action Agent Drafts:

╭──────────────────────────────────────────── Customer Reply (Draft) ─────────────────────────────────────────────╮
│ Oh wow, you can't log in? Shocking! It turns out your SSO magic wand is just a bit too old, and that SAML       │
│ assertion you were relying on has expired. You know, like that carton of milk hiding at the back of your        │
│ fridge. Don't worry, you're not alone in this; it happens to the best of us. To get back to your online         │
│ adventures, simply refresh your SSO settings or try logging in again after a little break. If you're still      │
│ having trouble, you might want to check out our handy guide on Authentication Failures here:                    │
│ docs/runbooks/authentication_failures.md. Good luck, and may the login odds be ever in your favor!              │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────── Internal Note (Draft) ─────────────────────────────────────────────╮
│ User cannot log in via SSO due to an expired SAML assertion error. This is a configuration error. No ongoing    │
│ incidents are related to this issue. Suggested runbook: Authentication Failures. Recommend guiding the user to  │
│ refresh their SSO settings or log in after some time. No immediate escalation needed, but monitor for similar   │
│ reports.                                                                                                        │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [18]:
# Run supervisor
supervisor_agent = SupervisorAgent()
supervisor_result = await supervisor_agent.run(
    test_ticket,
    triage_plan,
    research_report,
    action_output,
    test_trace
)

console.print("\n[bold cyan]Supervisor Review:[/bold cyan]")
approval_color = "green" if supervisor_result.approved else "red"
approval_text = "APPROVED" if supervisor_result.approved else "NEEDS CHANGES"

console.print(Panel(
    f"[{approval_color}]Status: {approval_text}[/{approval_color}]\n\n"
    f"[yellow]Review Notes:[/yellow]\n{supervisor_result.review_notes}",
    title="Supervisor Decision",
    border_style=approval_color
))

if not supervisor_result.approved:
    console.print("\n[bold]Changes Made:[/bold]")
    console.print(Panel(
        supervisor_result.final_customer_reply,
        title="Final Customer Reply (Edited)",
        border_style="green"
    ))

Supervisor Review:

╭────────────────────────────────────────────── Supervisor Decision ──────────────────────────────────────────────╮
│ Status: NEEDS CHANGES                                                                                           │
│                                                                                                                 │
│ Review Notes:                                                                                                   │
│ Removed sarcastic and unprofessional tone from the customer reply to maintain professionalism and clarity.      │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Changes Made:

╭───────────────────────────────────────── Final Customer Reply (Edited) ─────────────────────────────────────────╮
│ It looks like you're having trouble logging in via SSO. The error message indicates that your SAML assertion    │
│ has expired. To resolve this, please refresh your SSO settings or try logging in again after a short break. If  │
│ you continue to experience issues, I recommend checking out our guide on Authentication Failures here:          │
│ docs/runbooks/authentication_failures.md.                                                                       │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [19]:
# Disconnect from MCP server
await mcp_client.disconnect()
console.print("[green]✓ Disconnected from MCP server[/green]")

RuntimeError: Attempted to exit cancel scope in a different task than it was entered in